In [22]:
import numpy as np
import pandas as pd
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta

In [23]:
# Define your parameters
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5
initial_investment = 120000000
backup_amount_initial = 40_000_000


companies_vn30 = [
    'SSI', 'BCM','VHM','VIC','VRE','BVH','POW','GAS','ACB','BID',
'CTG','HDB','MBB','SSB','SHB','STB','TCB','TPB','VCB','VIB','VPB','HPG',
'GVR','MSN','VNM','SAB','VJC','MWG','PLX','FPT']

win_rate = 0.544579775
loss_rate = 1 - win_rate
mean_profit = 0.389644289
mean_loss = 0.1688279245


In [24]:
companies = ['VRE', 'HDB', 'ACB']

In [25]:
def calculate_indicators(df):
    if df.empty:
        return df
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Bollinger_high'] = ta.volatility.bollinger_hband(df['close'], window=15, window_dev=2)
    df['Bollinger_low'] = ta.volatility.bollinger_lband(df['close'], window=15, window_dev=2)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    return df

In [26]:
def kelly_criterion(p, q, profit, loss):
    return (p/loss) - (q/profit)

In [27]:
kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)

2.056839669042076

In [28]:
def  boll_rsi_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['close'] <= df['Bollinger_low']) &
        (df['RSI'] <= RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] >= RSI_OVERBOUGHT) &
        (df['close'] >= df['Bollinger_high']), 'Signal'] = -1

    return df

In [29]:
def simulate_investment(ticker, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    try:
        data = vn.stock_historical_data(ticker, '2021-01-01', '2024-01-01', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = boll_rsi_strategy(data)

        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        backup_amount = backup_amount_initial
        portfolio_values = []
        overspend = 0

        f_star = 0.25 * kelly_criterion(win_rate, loss_rate, mean_profit ,mean_loss)

        for i in range(len(data)):
            if data.index[i] in buy_signals:
                allocation = cash * f_star
                if allocation > 0:
                    shares_to_buy = int(allocation // data['close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            cash_needed = total_cost - cash
                            if cash_needed <= backup_amount:
                                cash = 0
                                backup_amount -= cash_needed
                                overspend += cash_needed
                            else: 
                                shares_to_buy = int((cash + backup_amount) // data['close'][i])
                                cash_needed = shares_to_buy * data['close'][i] - cash
                                cash = 0
                                backup_amount -= cash_needed
                                overspend += cash_needed
                        holdings += shares_to_buy
                        

            if data.index[i] in sell_signals and holdings > 0:
                current_price = data['close'].iloc[i]
                shares_to_sell = int(holdings * sell_fraction)
                if shares_to_sell > 0:
                    if shares_to_sell > holdings:
                        shares_to_sell = holdings
                    revenue = shares_to_sell * current_price 
                    cash += revenue
                    holdings -= shares_to_sell
                    # print(f"Selling {shares_to_sell} shares at {current_price} on {data.index[i]}")
                    #Check profit and repay backup amount if possible 
                    if cash > overspend:
                        repayment_amount = overspend
                        cash -= repayment_amount
                        overspend = 0
                        backup_amount += repayment_amount

            current_value = cash + holdings * data['close'].iloc[i] + backup_amount
            portfolio_values.append(current_value)
            # print(f"Day {data.index[i]}: Cash: {cash}, Holdings: {holdings}, Current Value: {current_value}")

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - (initial_investment+backup_amount)

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [30]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment+backup_amount_initial)  * 100
            })
    return pd.DataFrame(results)

In [31]:
sell_fraction = 0.25 * kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)

# Save results to CSV
results_df.to_csv('result/Boll_kelly.csv', index=False)

# Print results
print(results_df)

average_rate_of_return = results_df['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)

C:\Users\Admin\AppData\Local\Temp\ipykernel_308\1219599999.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_308\648791641.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = int(allocation // data['close'][i])
C:\Users\Admin\AppData\Loca

   Company  Final Portfolio Value  Total Profit  Rate of Return
0      SSI              191818384      31818384       19.886490
1      BCM              158776640      -1223360       -0.764600
2      VHM              134942120     -25057880      -15.661175
3      VIC              144081029     -15918971       -9.949357
4      VRE              165454800       5454800        3.409250
5      BVH              130644730     -29355270      -18.347044
6      POW              177886749      17886749       11.179218
7      GAS              224368530      64368530       40.230331
8      ACB              225195950      65195950       40.747469
9      BID              214889180      54889180       34.305737
10     CTG              222495610      62495610       39.059756
11     HDB              241797040      81797040       51.123150
12     MBB              166026500       6026500        3.766563
13     SSB              171147720      11147720        6.967325
14     SHB              140096104     -1

C:\Users\Admin\AppData\Local\Temp\ipykernel_308\1219599999.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_308\648791641.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = int(allocation // data['close'][i])
C:\Users\Admin\AppData\Loca